In [ ]:
import sys
print(sys.executable)

c:\Users\ygh04\Desktop\semiconductor_yield_project\venv\Scripts\python.exe


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [2]:
df = pd.read_csv('../data/uci-secom.csv')
print(df.head())

                  Time        0        1          2          3       4      5  \
0  2008-07-19 11:55:00  3030.93  2564.00  2187.7333  1411.1265  1.3602  100.0   
1  2008-07-19 12:32:00  3095.78  2465.14  2230.4222  1463.6606  0.8294  100.0   
2  2008-07-19 13:17:00  2932.61  2559.94  2186.4111  1698.0172  1.5102  100.0   
3  2008-07-19 14:43:00  2988.72  2479.90  2199.0333   909.7926  1.3204  100.0   
4  2008-07-19 15:22:00  3032.24  2502.87  2233.3667  1326.5200  1.5334  100.0   

          6       7       8  ...       581     582     583     584      585  \
0   97.6133  0.1242  1.5005  ...       NaN  0.5005  0.0118  0.0035   2.3630   
1  102.3433  0.1247  1.4966  ...  208.2045  0.5019  0.0223  0.0055   4.4447   
2   95.4878  0.1241  1.4436  ...   82.8602  0.4958  0.0157  0.0039   3.1745   
3  104.2367  0.1217  1.4882  ...   73.8432  0.4990  0.0103  0.0025   2.0544   
4  100.3967  0.1235  1.5031  ...       NaN  0.4800  0.4766  0.1045  99.3032   

      586     587     588       589  P

In [ ]:
df.info()

In [ ]:
print(df.describe())

In [ ]:
print(df['31'].isnull().sum())

In [ ]:
print(df['Pass/Fail'].value_counts())

In [ ]:
# 'Time'과 'Pass/Fail' 컬럼을 제외한 센서 데이터 특성들 선택
features = df.drop(columns=['Time', 'Pass/Fail'])

# 시작 인덱스를 지정하여 10개씩 반복해서 시각화
start_index = 5  # 이 값을 10, 20, 30 등으로 변경하여 다음 10개 특성을 볼 수 있습니다.
end_index = start_index + 10

# 서브플롯 생성: 5행 2열 구조
fig, axes = plt.subplots(nrows=5, ncols=2, figsize=(10, 15))
axes = axes.flatten()  # 2차원 배열을 1차원으로 변환하여 쉽게 접근

# 선택된 10개 특성에 대한 히스토그램 그리기
for i, col in enumerate(features.columns[start_index:end_index]):
    sns.histplot(data=features, x=col, kde=True, ax=axes[i])
    axes[i].set_title(f'Histogram of Feature {col}')
    axes[i].set_xlabel('Value')
    axes[i].set_ylabel('Frequency')

plt.tight_layout()  # 그래프 간 간격 자동 조절
plt.show()

In [ ]:

# 예시로 '0'번 특성(컬럼)의 박스플롯 확인
# '0' 대신 다른 특성 번호를 넣어 여러 컬럼 확인
feature_to_plot = '100'

plt.figure(figsize=(8, 6))
sns.boxplot(x=df[feature_to_plot])
plt.title(f'Boxplot of Feature {feature_to_plot}')
plt.xlabel(f'Feature {feature_to_plot} Value')
plt.show()

In [3]:
# 'Time' 컬럼과 'Pass/Fail' 컬럼 -> 제외하고 결측치 처리
features = df.drop(columns=['Time', 'Pass/Fail'])

# 각 특성(feature)의 중앙값 계산
median_values = features.median()

# 중앙값을 사용하여 결측치 채우기
df_filled = features.fillna(median_values)

In [4]:
# 결측치 모두 채워졌는지 확인
print(df_filled.isnull().sum().sum())

0


In [5]:
from sklearn.model_selection import train_test_split

# 독립 변수(X)와 종속 변수(y) 분리
X = df_filled
y = df['Pass/Fail']

# 훈련 및 테스트 데이터 분할
# 'stratify=y'를 사용해 클래스 비율을 유지
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, f1_score, recall_score, classification_report

# 1. 데이터 로드
print("--- 1. 데이터 로드 및 초기 탐색 ---")
df = pd.read_csv('../data/uci-secom.csv')
print("원본 데이터셋 크기:", df.shape)
print("Pass/Fail 컬럼의 고유값:", df['Pass/Fail'].unique())
print("\n" + "="*50 + "\n")


# 2. 데이터 전처리
print("--- 2. 데이터 전처리 (결측치 처리 및 레이블 변환) ---")
# 'Time' 컬럼과 'Pass/Fail' 컬럼 제외
features = df.drop(columns=['Time', 'Pass/Fail'])

# 각 특성(feature)의 중앙값 계산
median_values = features.median()

# 중앙값을 사용하여 결측치 채우기
df_filled = features.fillna(median_values)

# 'Pass/Fail' 컬럼의 -1 값을 0으로 변환하여 모델이 인식할 수 있게 함
y = df['Pass/Fail'].replace(-1, 0)
print("변환 후 Pass/Fail 컬럼의 고유값:", y.unique())

# 독립 변수(X)와 종속 변수(y) 분리
X = df_filled
print("\n" + "="*50 + "\n")


# 3. 훈련 및 테스트 데이터 분할
print("--- 3. 훈련 및 테스트 데이터 분할 ---")
# 'stratify=y'를 사용해 클래스 비율을 유지
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"훈련 데이터 크기: {X_train.shape}, 테스트 데이터 크기: {X_test.shape}")
print("훈련 데이터의 클래스 비율:\n", y_train.value_counts(normalize=True))
print("테스트 데이터의 클래스 비율:\n", y_test.value_counts(normalize=True))
print("\n" + "="*50 + "\n")


# 4. 초기 모델링 (Baseline Modeling)
print("--- 4. 초기 모델링 및 성능 평가 ---")

# 로지스틱 회귀 모델 훈련 및 평가
print("--- 로지스틱 회귀 모델 ---")
lr_model = LogisticRegression(random_state=42, solver='liblinear')
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_pred_proba = lr_model.predict_proba(X_test)[:, 1]

print(f"AUC: {roc_auc_score(y_test, lr_pred_proba):.4f}")
print(f"F1-Score: {f1_score(y_test, lr_pred):.4f}")
print(f"Recall: {recall_score(y_test, lr_pred):.4f}")
print("--- 성능 보고서 ---")
print(classification_report(y_test, lr_pred, zero_division=0))
print("\n" + "-"*50 + "\n")

# 랜덤 포레스트 모델 훈련 및 평가
print("--- 랜덤 포레스트 모델 ---")
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print(f"AUC: {roc_auc_score(y_test, rf_pred_proba):.4f}")
print(f"F1-Score: {f1_score(y_test, rf_pred):.4f}")
print(f"Recall: {recall_score(y_test, rf_pred):.4f}")
print("--- 성능 보고서 ---")
print(classification_report(y_test, rf_pred, zero_division=0))
print("\n" + "-"*50 + "\n")


# XGBoost 모델 훈련 및 평가
print("--- XGBoost 모델 ---")
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

print(f"AUC: {roc_auc_score(y_test, xgb_pred_proba):.4f}")
print(f"F1-Score: {f1_score(y_test, xgb_pred):.4f}")
print(f"Recall: {recall_score(y_test, xgb_pred):.4f}")
print("--- 성능 보고서 ---")
print(classification_report(y_test, xgb_pred, zero_division=0))
print("\n" + "="*50 + "\n")


# 최종 결론 출력
print("--- 최종 결과 분석 ---")
print("다양한 모델의 초기 성능을 비교하고, 불균형 데이터에 중요한 지표인 Recall, F1-Score, AUC를 확인합니다.")
print("이 결과를 바탕으로 데이터 불균형을 해결할 모델을 선정하고, 다음 단계로 나아갈 수 있습니다.")

--- 1. 데이터 로드 및 초기 탐색 ---
원본 데이터셋 크기: (1567, 592)
Pass/Fail 컬럼의 고유값: [-1  1]


--- 2. 데이터 전처리 (결측치 처리 및 레이블 변환) ---
변환 후 Pass/Fail 컬럼의 고유값: [0 1]


--- 3. 훈련 및 테스트 데이터 분할 ---
훈련 데이터 크기: (1253, 590), 테스트 데이터 크기: (314, 590)
훈련 데이터의 클래스 비율:
 Pass/Fail
0    0.933759
1    0.066241
Name: proportion, dtype: float64
테스트 데이터의 클래스 비율:
 Pass/Fail
0    0.933121
1    0.066879
Name: proportion, dtype: float64


--- 4. 초기 모델링 및 성능 평가 ---
--- 로지스틱 회귀 모델 ---


c:\Users\ygh04\Desktop\semiconductor_yield_project\venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


AUC: 0.6709
F1-Score: 0.0667
Recall: 0.0476
--- 성능 보고서 ---
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       293
           1       0.11      0.05      0.07        21

    accuracy                           0.91       314
   macro avg       0.52      0.51      0.51       314
weighted avg       0.88      0.91      0.89       314


--------------------------------------------------

--- 랜덤 포레스트 모델 ---
AUC: 0.7538
F1-Score: 0.0000
Recall: 0.0000
--- 성능 보고서 ---
              precision    recall  f1-score   support

           0       0.93      1.00      0.97       293
           1       0.00      0.00      0.00        21

    accuracy                           0.93       314
   macro avg       0.47      0.50      0.48       314
weighted avg       0.87      0.93      0.90       314


--------------------------------------------------

--- XGBoost 모델 ---


c:\Users\ygh04\Desktop\semiconductor_yield_project\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [23:13:45] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


AUC: 0.7062
F1-Score: 0.0000
Recall: 0.0000
--- 성능 보고서 ---
              precision    recall  f1-score   support

           0       0.93      1.00      0.97       293
           1       0.00      0.00      0.00        21

    accuracy                           0.93       314
   macro avg       0.47      0.50      0.48       314
weighted avg       0.87      0.93      0.90       314



--- 최종 결과 분석 ---
다양한 모델의 초기 성능을 비교하고, 불균형 데이터에 중요한 지표인 Recall, F1-Score, AUC를 확인합니다.
이 결과를 바탕으로 데이터 불균형을 해결할 모델을 선정하고, 다음 단계로 나아갈 수 있습니다.
